# Introduction to rustfuzz

`rustfuzz` is a blazing fast string matching library implemented in Rust. It provides a simple Python API that heavily mirrors the popular RapidFuzz, but powered by Rust for optimal performance.

Let's start with basic matching!

## Basic String Matching (`fuzz_ratio` and `fuzz_partial_ratio`)

The simplest way to compare two strings is using `fuzz_ratio`, which calculates the standard Levenshtein distance ratio.

If you want to compare a short string against a longer string, `fuzz_partial_ratio` is optimal. It finds the best matching substring of the longer string.

In [ ]:
from rustfuzz.fuzz import fuzz_ratio, fuzz_partial_ratio

# Exact match
print(f"Ratio (exact): {fuzz_ratio('apple', 'apple')}")

# Small edit
print(f"Ratio (edit): {fuzz_ratio('apple', 'bpple')}")

# Partial match against longer string
print(f"Partial Ratio: {fuzz_partial_ratio('apple', 'I love to eat an apple every day!')}")